# Śledzenie obiektów

<img src="https://i.imgur.com/wKXXFkQ.png" width="500">

## Wstęp
W erze cyfrowej, w obliczu rosnącej lawinowo ilości danych wideo, zdolność do ich automatycznego rozpoznawania i interpretowania staje się kluczowa w wielu dziedzinach – od bezpieczeństwa publicznego po autonomiczne pojazdy. Technologie oparte na głębokim uczeniu rewolucjonizują sposób, w jaki przetwarzamy informacje wizualne. Kluczowym wyzwaniem jest tu detekcja i śledzenie obiektów na filmach wideo.

Celem tego zadania jest opracowanie algorytmu, który będzie w stanie analizować sekwencje ruchów w grze "trzy kubki". Uczestnicy mają za zadanie określić końcową pozycję kubków po serii ruchów, korzystając z analizy statycznych obrazów z każdej klatki nagrania.

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Poniższe funkcje ułatwiają pracę z dostarczonymi danymi
# W kolejnych komórkach zobaczysz przykłady ich użycia
from utils.utils import get_level_info, get_video_data, display_video, download_and_replace_data

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

images, coordinates, target, path_to_images = get_video_data(level=1,video_id=0,dataset="example")
display_video(images,rescale=0.7,FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

ModuleNotFoundError: No module named 'utils'

## Zadanie 1: Gra w trzy kubki

Jedną z możliwości podejścia do problemu rozpoznawania obiektów na wideo jest zastosowanie modelu dedykowanego do analizy statycznych obrazów dla każdej z klatek. To właśnie będziemy starali się tutaj osiągnąć. Dla każdej klatki w animacjach dostarczyliśmy przewidziany przez model opis tego, w których miejscach znajdują się kubki. Na nagraniach pokazane jest jak są zamieniane miejscami. Twoim zadaniem będzie określenie pozycji, na którą finalnie trafią. Oznaczymy ustawienie początkowe jako $[0,1,2]$ licząc w kolejności od lewej do prawej (po współrzędej `x`). Jeżeli następnie przestawimy pierwszy od lewej kubek na przeciwległy koniec, uzyskamy $[1,2,0]$.

Będziesz miał dostęp zarówno do wszystkich klatek animacji, jak i do oznaczonych przez nas prostokątów ograniczających, w których znajdują się kubki. Co ważne, algorytm, który będziesz tworzył ma korzystać jedynie z informacji o prostokątach ograniczających. W tym zadaniu, klatki wideo są dostarczone jedynie do wizualizacji przykładów i algorytmu, na własne potrzeby.

Punkty za to zadanie będą przyznane za osiągnięcie jak najdokładniejszych predykcji na zbiorze testowym. Kryterium będzie *accuracy*. Ewaluacja na zbiorze testowym będzie dokonana przez organizatorów.

## Pliki zgłoszeniowe
Tylko ten notebook zawierający **kod** oraz **krótki raport** opisujący Twoje rozwiązanie (do 300 słów). Miejsce na raport znajdziesz na końcu tego notebooka.

## Ograniczenia
- Twoja funkcja powinna zwracać predykcje w maksymalnie 5 minut używając Google Colab bez GPU.

## Uwagi i wskazówki
- Testuj swoje rozwiązanie na zbiorze plików wideo `level_1`.
- **Skuteczność modelu**: przetestuj skuteczność modelu na zbiorze walidacyjnym używając dostarczonej przez nas funkcji **submission_script**, umieść ten wynik w raporcie.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to podzadanie możesz zdobyć pomiędzy 0 i 0.5 punktów. Zdobędziesz 0 punktów jeśli Twoje accuracy na zbiorze testowym będzie poniżej 50%. Jeśli będzie równe 100%, otrzymasz 0.5 punktu. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością metryki.

# Kod startowy

In [ ]:
# Poniższe biblioteki są wystarczające do wykonania wszystkich zadań
# Jeśli jednak chcesz użyć innych, sprawdź czy są dostępne na serwerze (requirements.txt)
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import IPython.display
import json
import PIL
import sklearn as sk

In [ ]:
# Funkcja pomocnicza do ładowania danych
images, _, _, _ = get_video_data(level=1,video_id=0,dataset="example")

with open(os.path.join(os.getcwd(),'example_tracks','tracks_1_0.json'), 'r') as f:
    tracks = json.load(f)

for key in tracks.keys():
    tracks[key] = [tuple(el) for el in tracks[key]]

# Funkcja pomocnicza do wyświetlania danych
display_video(images,
                tracks=tracks,
                rescale=0.7,
                FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [ ]:
# Pobieranie danych do podzadań 1, 2 i 3 (około ~646Mb), skrypt będzie wykonywał się parę minut
# Wystarczy, że pobierzesz dane tylko raz. Na serwerze sprawdzającym dane będą już pobrane
# Struktura plików będzie identyczna jak tutaj
if not FINAL_EVALUATION_MODE:
    download_and_replace_data()

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# Funkcja pomocnicza do testowania algorytmu
def submission_script(algorithm,level,verbose=False,dataset="valid"):
    num_videos, _ = get_level_info(level=level,dataset=dataset)
    correct = []
    exception_messages = set()
    for video_number in range(num_videos):
        _, coordinates, target, _ = get_video_data(level=level,video_id=video_number,dataset=dataset)
        try:
            prediction = algorithm(coordinates)
            if tuple(target) == tuple(prediction):
                correct.append(1)
            else:
                correct.append(0)
            if verbose:
                print(f"Video: animation_{str(video_number).zfill(4)}")
                print(f"Prediction: {prediction}")
                print(f"Target:     {target}")
                print(f"Score: {tuple(target) == tuple(prediction)}", end='\n\n')
        except Exception as e:
            correct.append(0)
            exception_messages.add(str(e))
    if verbose:
        print(f"Accuracy: {np.mean(correct)}")
        print(f"Correctness: {correct}")
    return np.sum(correct) / num_videos, correct, exception_messages

# Twoje rozwiązanie

In [ ]:
def your_algorithm_task_1(coordinates): # nie zmieniaj nazwy funkcji
    #TODO - zaimplementuj swój algorytm

    permutation = [0,1,2]
    return permutation

In [ ]:
# Sprawdź jak działa Twój algorytm
accuracy, correctness, _ = submission_script(
    algorithm=your_algorithm_task_1,
    level=1,
    verbose=True,
    dataset="train")

In [ ]:
# Zapisz swój raport do zmiennej poniżej, abyśmy mogli go później automatycznie odczytać sprawdzaczką
raport_1 = \
"""
Raport z zadania:
.
.
.
"""